In [2]:
import psycopg2

In [3]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

sql = '''
select
    route_id, 
    stop_id_start, 
    stop_id_end,
    to_char(max(end_stop_time - init_stop_time), 'HH24:MI:SS') as diff_time
from
    warehouse
group by cube (route_id, stop_id_start, stop_id_end)
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results:
    if row[1] == None and row[2] == None:
        print(f'route {row[0]}, max_time {row[-1]}h')

route 106, max_time 00:13:00h
route 107, max_time 00:25:00h
route 10M, max_time 00:25:00h
route 11M, max_time 00:25:00h
route 12M, max_time 00:28:00h
route 13M, max_time 00:27:00h
route 1M, max_time 00:25:00h
route 200, max_time 00:44:00h
route 201, max_time 00:53:00h
route 202, max_time 00:54:00h
route 203, max_time 00:51:00h
route 204, max_time 01:05:30h
route 205, max_time 01:03:00h
route 206, max_time 00:48:08h
route 207, max_time 01:00:00h
route 208, max_time 00:52:00h
route 209, max_time 00:36:00h
route 300, max_time 01:19:00h
route 301, max_time 01:27:00h
route 302, max_time 00:41:55h
route 303, max_time 00:52:10h
route 304, max_time 00:27:00h
route 305, max_time 00:41:11h
route 3M, max_time 00:28:00h
route 400, max_time 00:42:00h
route 401, max_time 00:30:00h
route 402, max_time 00:35:05h
route 403, max_time 00:40:00h
route 4M, max_time 00:25:00h
route 500, max_time 00:46:00h
route 501, max_time 00:59:18h
route 502, max_time 00:47:04h
route 503, max_time 01:01:33h
route 504, ma

In [4]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

sql = '''
select
    route_id, 
    to_char(init_stop_time, 'HH24:MI:SS') as init_time,
    count(*)
from
    warehouse 
group by cube (route_id, init_stop_time)
order by count(*) desc
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results[:5]:
    print(row)

(None, None, 1434319)
('207', None, 108461)
('500', None, 68674)
('206', None, 67052)
('403', None, 62229)


In [11]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# delay: how much time after the taxi would be taken did the bus arrive
sql = '''
select
    trip_id,
    to_char(initial_ts, 'HH24:MI:SS') as i_ts,
    to_char(init_stop_time, 'HH24:MI:SS') as i_stop_time,
    to_char(max(init_stop_time - initial_ts), 'HH24:MI:SS') as diff_time
from
    warehouse 
group by cube (trip_id, initial_ts, init_stop_time)
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results[:5]:
    print(f'trip_id {row[0]}, delay {row[-1]}h')

trip_id 106_0_U_10, delay 00:10:53h
trip_id 106_0_U_10, delay 00:10:53h
trip_id 106_0_U_10, delay 00:10:53h
trip_id 106_0_U_14, delay 00:27:46h
trip_id 106_0_U_14, delay 00:27:46h
